## 最初の大きなプロジェクト - プロフェッショナルなあなた！

### 概要 ... そして、ツールの使用

AIエージェントを使ったキャリアアバター（自己紹介AIエージェント）の構築・運用・デプロイ方法を解説

- Pushoverの導入
  - Pushoverというサービスを使って、スマホにプッシュ通知を送る仕組みを構築
  - `.env`ファイルでPushoverのユーザーキーとトークンを設定

- 必要なライブラリのインポート  
- Pushover用の初期化と動作確認  
- ユーザー情報・未知の質問の記録用関数のツール化
- **Function Callingでツール呼出処理（if分岐／globals動的呼び出し）**
- LinkedInのPDFとサマリーtxtファイルからプロフィール情報の取得
- AIが「Ed Donnerとして」質問に回答するためのシステムプロンプトの作成
- システムプロンプトにプロフィール情報を含めWeb上のやりとりを本人として再現
- Gradioを使用してWebベースの対話型チャットの実装
- HuggingFace Spacesへのデプロイ方法の説明
- 応用・商用展開・エクササイズ
  - RAG（検索拡張生成）やツール追加例
  - 商用応用（専門領域のAIアシスタント）

### しかし、最初に：Pushoverの紹介

Pushoverは、プッシュ通知を携帯電話に送信するための気の利いたツールです。

セットアップとインストールは非常に簡単です！

https://pushover.net/ にアクセスして、右上の「ログインまたはサインアップ」をクリックして無料のアカウントにサインアップし、APIキーを作成します。

サインアップしたら、ホーム画面で "Create an Application/API Token" をクリックし、名前（エージェントなど）を指定し、[アプリケーションの作成]をクリックします。

次に、 `.env` ファイルに2行を追加します。

- PUSHOVER_USER=_Pushoverのホーム画面の右上にある、おそらくu_で始まるキーを入力
- PUSHOVER_TOKEN=_キーを入力して、エージェント（または何でも）という新しいアプリケーションをクリックして、おそらくa_で始まるキーを入力

`.env`ファイルを保存し、` load_dotenv（override = true） `を保存した後、環境変数を設定することを忘れないでください。

最後に、「電話、タブレット、またはデスクトップの追加」をクリックして、電話にインストールします。

In [1]:
# import

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
# 通常のスタート

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# Pushover用

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"{pushover_user[0]}で始まるPushoverのユーザーが見つかりました")
else:
    print("プッシュオーバーユーザーが見つかりません")

if pushover_token:
    print(f"{pushover_token[0]}で始まるPushoverのトークンが見つかりました")
else:
    print("プッシュオーバートークンが見つかりません")

uで始まるPushoverのユーザーが見つかりました
aで始まるPushoverのトークンが見つかりました


In [4]:
# プッシュ通知の関数
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
# プッシュ通知の送付
push("HEY!!")

Push: HEY!!


In [6]:
# ユーザーの詳細を記録する
def record_user_details(email, name="Name not provided", notes="not provided"):
    # {name} からの関心を、メールアドレス {email}、メモ {notes} で記録
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

# 不明な質問を記録する
def record_unknown_question(question):
    # 答えられない質問を記録
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [7]:
# Function Callingする関数に対応するJSONのメタデータの定義
# フレームワークを使用すればdocstringなどを使用して自動化可能 → LlamaIndex3等を参照）

# このツール（record_user_details）を使用すると、ユーザーの連絡用メールアドレス（や名前、ノート）を記録。
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

# 答えがわからず回答できなかった質問は必ずこのツール（record_unknown_question）を使用して記録してください。
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [8]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [9]:
# IFステートメントでFunction Calling
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:

        # この辺り、構造化出力風
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [10]:
# レイドバインドで関数呼び出し
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [11]:
# レイトバインド風にFunction Calling
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:

        # この辺り、構造化出力風
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # レイトバインド風実装!!!
        
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [12]:
# キャリアアバター（経歴紹介AI）の準備：外部リソース

reader = PdfReader("me/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [13]:
# キャリアアバター（経歴紹介AI）の準備：システムプロンプト
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [14]:
# chat関数
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    # ループで複数のツール呼び出しにも対応
    done = False
    while not done:

        # LLMへの呼び出し - ツールJSON（tools）を渡す
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        # finish_reasonをチェック
        finish_reason = response.choices[0].finish_reason
        
        # finish_reasonが"tool_calls"ならツールを呼び出す
        if finish_reason=="tool_calls": # "tool_calls" → ツール呼び出し
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)

            # 会話履歴に「LLM がツールを使おうとしたこと」を残す
            messages.append(message)    # 1要素
            # 次の呼び出しでLLM に「ツールの実行結果」を渡す
            messages.extend(results)    # n要素
        else:                           # "stop" → 通常の応答
            done = True

    # 最終応答
    return response.choices[0].message.content

In [15]:
# プロトタイプをテストする。
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


### 以下のようなことをChatで質問する
- hi there
- what is your current job?
- do you have a patent?
- who is your favorite musician?
- I'd like to get in touch!
- I'm at ed@edwarddonner.com

## そして今、デプロイ

このコードは（プロトタイプ実装を詰め込んだ）「app.py」にあり、コレを「HuggingFace Spaces」にデプロイします。

In [16]:
# app.pyをローカル実行（.envが使われる）
from app import Me
import gradio as gr

me = Me()
gr.ChatInterface(me.chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


#### 開始する前に
「me」ディレクトリでファイルがあなたについて話すように更新することを忘れないでください - LinkedInプロフィールとsummary.txt - また、`app.py` 中の `self.name = "Ed Donner"` も変更します。また、1_Foundationsディレクトリ内にREADMEファイルがないことを確認してください。ある場合は、削除してください。展開プロセスは、このディレクトリに新しいREADMEファイルを作成します。

#### デプロイの手順

1. https://huggingface.co にアクセスして、アカウントを設定。
1. 右上のアバターメニューから、アクセストークンを選択し「新しいトークンの作成」を選択し。書き込み許可を与えます（書き込み許可が必要です！）新しいキーの記録を保持。
1. ターミナルで `uv tool install 'huggingface_hub[cli]'` を実行し、uv を使って Hugging Face Hub ライブラリと CLI をインストール（コレは仮想環境で無い方が良い）。
1. そして、`hf auth login` でキーを使用してログイン。その後、`hf auth whoami` を実行しログインを確認（`hf` は `huggingface-cli` のエイリアス）。
1. 新しいトークンを取り、.envファイルに追加： `HF_TOKEN=hf_xxx`
1. 1_Foundationsフォルダから `uv run gradio deploy` と入力 ... コレは、「Hugging Face Spaces」や「Gradio Spaces」にGradioのUIアプリをデプロイするためのサブコマンド。
1. なお、基本的に、1_Foundationsフォルダのカレントとサブフォルダ中の除外の条件に引っかからないファイルがアップロードされるもよう。
1. 指示に従い、"career_conversation"に名前を付け、`app.py` を指定し、ハードウェアとしてcpu-basicを選択し、
1. `.env` はデプロイされないので、別途シークレットを入力（OpenAI APIキー、Pushoverユーザー、トークン）
1. 最後に GitHub Action に関する問いに「いいえ」と答える。

これらのヒントをありがとう、ロバート、ジェームズ、マーティンズ、アンドラス、プリヤに感謝します。  
次の2つのセクションをお読みください - 秘密を変更する方法、およびスペースを再配置する方法（この1_Foundationsディレクトリで作成されるreadme.mdを削除する必要がある場合があります）。

#### シークレットの詳細

デプロイ後もシークレットを変えることができる。

1. HuggingfaceのWebサイトにログイン
1. 右上のアバターメニューからプロフィールに移動
1. デプロイしたSpacesを選択
1. 右上の設定歯車をクリックして下にスクロールして、
1. シークレットを変更、スペースを再起動・削除など。

#### これで配備が完了しました!

すべてを完全に交換する場合は、1_Foundationsフォルダで作成されたREADMEファイルを削除する必要がある場合があります。

これが私のものです: https://huggingface.co/spaces/ed-donner/Career_Conversation

学生がどのように彼らが彼らの国の大統領になることができるかを尋ねたというプッシュ通知を受け取りました😂😂

展開の詳細については：

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

スペースを削除するには：  

1. HuggingfaceのWebサイトにログイン
1. 右上のアバターメニューからプロフィールに移動
1. デプロイしたSpacesを選択、右上の設定歯車をクリック
1. 下にスクロールして、スペースを削除
1. また、Gradio がこの 1_foundations フォルダ内に作成した README ファイルを削除 (そうしないと、次回 Gradio をデプロイするときに質問が表示されなくなります)。

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">演習</h2>
            <span style="color:#ff7800;">
                • まず第一に、これを実際に自分で使ってみましょう！これは本当に価値のあるツールであり、将来の履歴書にもなります。<br/>
                • 次に、リソースを改善しましょう。自分自身に関するより良いコンテキストを追加しましょう。RAG について知っているなら、自分に関するナレッジベースを追加しましょう。<br/>
                • ツールをさらに追加しましょう！LLM が読み書きできる、よくある Q&A をまとめた SQL データベースを作成することも可能です。<br/>
                • 前回のラボで使用した Evaluator を取り込み、他の Agentic パターンを追加しましょう。
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">商業的影響</h2>
            <span style="color:#00bfff;">
                明らかなこと（キャリアにおけるもう一つの人格）以外にも、専門知識を持ち、現実世界とインタラクションできるAIアシスタントが必要なあらゆる状況において、ビジネスに応用できます。
            </span>
        </td>
    </tr>
</table>